In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(dplyr)
    library(tibble)
})


In [1]:
source("scTools/R/process.geo.data.r")

In [6]:
seurat_objs <- process.GEO.data(GEOID = "GSE134809")
length(seurat_objs)
seurat_objs

[1] 31

$GSM397200969
An object of class Seurat 
15951 features across 3421 samples within 1 assay 
Active assay: RNA (15951 features, 0 variable features)
 1 layer present: counts

$GSM397201068
An object of class Seurat 
15982 features across 8123 samples within 1 assay 
Active assay: RNA (15982 features, 0 variable features)
 1 layer present: counts

$GSM3972011122
An object of class Seurat 
15225 features across 1438 samples within 1 assay 
Active assay: RNA (15225 features, 0 variable features)
 1 layer present: counts

$GSM3972012123
An object of class Seurat 
15386 features across 2725 samples within 1 assay 
Active assay: RNA (15386 features, 0 variable features)
 1 layer present: counts

$GSM3972013128
An object of class Seurat 
17085 features across 2964 samples within 1 assay 
Active assay: RNA (17085 features, 0 variable features)
 1 layer present: counts

$GSM3972014129
An object of class Seurat 
16012 features across 2378 samples within 1 assay 
Active assay: RNA (16012 features,

In [7]:
# Merge samples
seurat_objs <- get(load("objects/GSE134809.rda"))
obj <- merge(seurat_objs[[1]], 
c(seurat_objs[[2]], seurat_objs[[3]], seurat_objs[[4]], seurat_objs[[5]], seurat_objs[[6]], seurat_objs[[7]],
  seurat_objs[[8]], seurat_objs[[9]], seurat_objs[[10]], seurat_objs[[11]], seurat_objs[[12]], seurat_objs[[13]],
  seurat_objs[[14]], seurat_objs[[15]], seurat_objs[[16]], seurat_objs[[17]], seurat_objs[[18]], 
  seurat_objs[[19]], seurat_objs[[20]], seurat_objs[[21]], seurat_objs[[22]], seurat_objs[[23]], 
  seurat_objs[[24]], seurat_objs[[25]], seurat_objs[[26]], seurat_objs[[27]], seurat_objs[[28]], 
  seurat_objs[[29]], seurat_objs[[30]], seurat_objs[[31]]))

# Update metadata
obj@meta.data$Stim <- obj@meta.data$characteristics_ch1.1
obj@meta.data$Tissue <- obj@meta.data$tissue.ch1
obj@meta.data$Donor <- obj@meta.data$title

columns_to_keep <-  c('Donor', 'Tissue', 'Stim')
obj@meta.data <- obj@meta.data[, columns_to_keep, drop = FALSE]
head(obj@meta.data)
obj
save(obj, file = file.path("objects", "00.rda"))

Warning message:
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


An object of class Seurat 
23022 features across 142663 samples within 1 assay 
Active assay: RNA (23022 features, 0 variable features)
 31 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, counts.6, counts.7, counts.8, counts.9, counts.10, counts.11, counts.12, counts.13, counts.14, counts.15, counts.16, counts.17, counts.18, counts.19, counts.20, counts.21, counts.22, counts.23, counts.24, counts.25, counts.26, counts.27, counts.28, counts.29, counts.30, counts.31

In [ ]:
# Get unique elements from each column
unique_elements <- lapply(obj@meta.data, unique)

# Print the unique elements for each column
print(unique_elements)

In [ ]:
# load(file=file.path("objects", "00.rda"))
options(future.globals.maxSize = 8000 * 1024^2)
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi',
conserve.memory = FALSE, verbose = TRUE)

save(obj, file=file.path("objects", "01.rda"))

In [8]:
# Clean up data
# obj <- JoinLayers(obj)
obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
# obj <- subset(obj, subset = nFeature_RNA > 350 & nFeature_RNA < 5000 & percent.mt < 10)
obj
save(obj, file = file.path("objects", "02.rda"))

An object of class Seurat 
23022 features across 115701 samples within 1 assay 
Active assay: RNA (23022 features, 0 variable features)
 1 layer present: counts

In [ ]:
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()

# Blueprint annnotation
sce <- LayerData(obj)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj@meta.data$blueprint.main <- blueprint.main$pruned.labels
save(obj, file=file.path("objects", "03.rda"))

# Monaco annontation
sce <- LayerData(obj)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj@meta.data$monaco.fine <- monaco.fine$pruned.labels
save(obj, file=file.path('objects', '04.rda'))

In [2]:
load(file=file.path("objects", "04.rda"))

In [5]:
obj

An object of class Seurat 
44058 features across 142663 samples within 2 assays 
Active assay: SCT (21036 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA

In [3]:
seu =  obj
seu
seu[["SCT"]] <- as(object = seu[["SCT"]], Class = "Assay")
seu

An object of class Seurat 
44058 features across 142663 samples within 2 assays 
Active assay: SCT (21036 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA

Warning message:
“Assay SCT changing from SCTAssay to Assay”


An object of class Seurat 
44058 features across 142663 samples within 2 assays 
Active assay: SCT (21036 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA

In [10]:
DefaultAssay(obj) <- "RNA"
obj <- JoinLayers(obj)

In [4]:


seu = JoinLayers(seu)
scConf1 = createConfig(seu)

ERROR: Error in UseMethod(generic = "JoinLayers", object = object): no applicable method for 'JoinLayers' applied to an object of class "c('Assay', 'KeyMixin')"


In [ ]:
load(file=file.path("objects", "04.rda"))
DefaultAssay(obj) <- "RNA"
obj <- JoinLayers(obj)
DefaultAssay(obj) <- "SCT"
obj <- subset(obj, subset = Tissue == "Blood")
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
# save(obj, file=file.path("objects", "03.rda"))

seu =  obj
scConf1 = createConfig(seu)
makeShinyFiles(seu, scConf1, gene.mapping = TRUE, shiny.prefix = "sc1", shiny.dir = "crohns_scRNAseq_GSE134809/", gex.assay = "SCT")

In [ ]:
load(file=file.path("objects", "04.rda"))
DefaultAssay(obj) <- "RNA"
obj <- JoinLayers(obj)
DefaultAssay(obj) <- "SCT"
obj <- subset(obj, subset = Tissue == "ileal")
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
# save(obj, file=file.path("objects", "03.rda"))

seu =  obj
scConf2 = createConfig(seu)
makeShinyFiles(seu, scConf2, gene.mapping = TRUE, shiny.prefix = "sc2", shiny.dir = "crohns_scRNAseq_GSE134809/", gex.assay = "SCT")